In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from models.gpt_model import GPTModel
from models.zeroshot_models import BartModel_withDescription
from models.zeroshot_models import BartModel_withParaphrasing
from models.zeroshot_models import DistilBertModel
from models.zeroshot_models import DistilBertModel_withParaphrasing
from models.zeroshot_models import MDeBertaModel
from models.zeroshot_models import MDeBertaModel_withParaphrasing
from models.zeroshot_models import MDeBertaModelMulti
from models.zeroshot_models import MDeBertaModelMulti_withParaphrasing
from models.tfidf_models import SVCModel
from models.tfidf_models import LogisticRegressionModel
from models.tfidf_models import RandomForestModel
from models.tfidf_models import KNNModel

In [2]:
test_data = pd.read_csv('data/test_data.csv' , sep=';')
X_test = test_data['text']
y_test = test_data['label']

# Dictionnaire pour stocker les performances des différents modèles
model_performance = {}

# Liste des modèles que vous avez déjà entraînés
models = {
    # "GPTModel": GPTModel(),
    # "BartModel_withDescription": BartModel_withDescription(),
    # "BartModel_withParaphrasing": BartModel_withParaphrasing(),
    # "DistilBertModel" : DistilBertModel(),
    # "DistilBertModel_withParaphrasing" : DistilBertModel_withParaphrasing(),
    # "MDeBertaModel" : MDeBertaModel(),
    # "MDeBertaModel_withParaphrasing" : MDeBertaModel_withParaphrasing(),
    # "MDeBertaModelMulti" : MDeBertaModelMulti(),
    # "MDeBertaModelMulti_withParaphrasing" : MDeBertaModelMulti_withParaphrasing()
    "SVCModel" : SVCModel(),
    "LogisticRegressionModel" : LogisticRegressionModel(),
    "RandomForestModel" : RandomForestModel(),
    "KNNModel" : KNNModel()

}

In [3]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, fbeta_score

def evaluate_model_performance(model, X_test, y_test, target_class='lost_luggage'):

    # Prédictions sur les données de test
    y_pred = model.predict(list(X_test))
    
    # Évaluation générale
    accuracy = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    
    # Calcul de la matrice de confusion complète
    labels = test_data['label'].unique()
    conf_matrix = confusion_matrix(y_test, y_pred, labels=labels)
    target_index = list(labels).index(target_class)
    
    # Précision pour `lost_luggage`
    # Précision = TP / (TP + FP)
    tp = conf_matrix[target_index, target_index]
    fp = conf_matrix[:, target_index].sum() - tp
    precision_lost_luggage = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    # Spécificité pour `lost_luggage`
    # Spécificité = TN / (TN + FP)
    tn = conf_matrix.sum() - (conf_matrix[target_index, :].sum() + conf_matrix[:, target_index].sum() - tp)
    specificity_lost_luggage = tn / (tn + fp) if (tn + fp) > 0 else 0


    # Évaluation pondérée
    f2_score_lost_luggage = fbeta_score(y_test, y_pred, beta=2, average='macro')
    
    # Résultats
    results = {
        "Overall Accuracy": accuracy,
        "F1 Score (Macro)": f1_macro,
        "Precision for lost_luggage": precision_lost_luggage,
        "Specificity for lost_luggage": specificity_lost_luggage,
        "F2 Score for lost_luggage": f2_score_lost_luggage,
    }
    
    return results

# Exécuter le pipeline de test pour chaque modèle
for model_name, model in models.items():
    print(f"Évaluation du modèle : {model_name}")
    performance = evaluate_model_performance(model, X_test, y_test)
    model_performance[model_name] = performance

# créer un DataFrame à partir des performances des modèles

model_performance_df = pd.DataFrame(model_performance).T
model_performance_df

Évaluation du modèle : SVCModel
Évaluation du modèle : LogisticRegressionModel
Évaluation du modèle : RandomForestModel
Évaluation du modèle : KNNModel


,Overall Accuracy,F1 Score (Macro),Precision for lost_luggage,Specificity for lost_luggage,F2 Score for lost_luggage
SVCModel,0.971429,0.969970,1.000000,1.000000,0.969864
LogisticRegressionModel,0.971429,0.970303,1.000000,1.000000,0.969984
RandomForestModel,0.946032,0.944036,0.966667,0.996491,0.944150
KNNModel,0.926984,0.926437,1.000000,1.000000,0.925181


In [5]:
#calculer pour chaque modele le score :  (F1_Score + (Precision for lost_luggage + Specificity for lost_luggage)/2 )/2

model_performance_df['final'] = (model_performance_df['F1 Score (Macro)'] + (model_performance_df['Precision for lost_luggage'] + model_performance_df['Specificity for lost_luggage'])/2 )/2

gpt : 55s

In [4]:
# sauvegarder les performances des modèles
model_performance_df.to_csv('test_results/models_performance_Tf-Idf.csv')

In [ ]:
# Matrice de confusion pour le modèle avec la meilleure performance

best_model_name = max(model_performance, key=lambda x: (model_performance[x]['Specificity for lost_luggage']+model_performance[x]['Precision for lost_luggage'])/2)
best_model = models[best_model_name]
y_pred_best = best_model.predict(X_test)

# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred_best, labels=test_data['label'].unique())
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_data['label'].unique(), yticklabels=test_data['label'].unique())
plt.xlabel("Prédictions")
plt.ylabel("Vérités")
plt.title(f"Matrice de Confusion pour le modèle {best_model_name}")
plt.show()

AttributeError: 'int' object has no attribute 'predict'